In [76]:
%pip install requests
%pip install PyGithub
%pip install python-gitlab
%pip install gitpython
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [105]:

import requests
from github import Github
from gitlab import Gitlab
from dotenv import load_dotenv
import os


load_dotenv()

# Configuration
GITLAB_URL = os.getenv('GITLAB_URL')
GITLAB_TOKEN = os.getenv('GITLAB_TOKEN')
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
GITLAB_PROJECT_ID = os.getenv('GITLAB_PROJECT_ID')
GITHUB_REPO_NAME = os.getenv('GITHUB_REPO_NAME')

# Initialize GitLab and GitHub clients
gitlab_client = Gitlab(GITLAB_URL, private_token=GITLAB_TOKEN)
github_client = Github(GITHUB_TOKEN)
# Get GitLab project
gitlab_project = gitlab_client.projects.get(GITLAB_PROJECT_ID)

# Create GitHub repository
user = github_client.get_user()


In [106]:
# Check if the repository already exists and delete it if it does
try:
    github_repo = github_client.get_repo(GITHUB_REPO_NAME)
    github_repo.delete()
    print(f"Deleted existing repository: {GITHUB_REPO_NAME}")
except Exception as e:
    print(f"Repository {GITHUB_REPO_NAME} does not exist or could not be deleted: {e}")

# Create a new GitHub repository
github_repo = user.create_repo(GITHUB_REPO_NAME, private=True)
repo_url = gitlab_project.http_url_to_repo.replace('https://', f'https://oauth2:{GITLAB_TOKEN}@')


Repository mynetcoreproject does not exist or could not be deleted: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest", "status": "404"}


In [107]:
import os
from git import Repo


# Migrate repository content
# Clone the GitLab repository
repo_name = GITLAB_PROJECT_ID.split('/')[-1]
if not os.path.exists(repo_name):
    Repo.clone_from(repo_url, repo_name, mirror=True)


# Push the repository to GitHub
github_repo_url = f'https://{GITHUB_TOKEN}@github.com/{user.login}/{GITHUB_REPO_NAME}.git'
repo = Repo(repo_name)
repo.remotes.origin.set_url(github_repo_url)
repo.remotes.origin.push(mirror=True)

print("Repository content migrated successfully.")


Repository content migrated successfully.


In [115]:
# Migrate open pull requests
import json
merge_requests = gitlab_project.mergerequests.list(state='opened')
with open('usermapping.json', 'r') as file:
    username_mapping = json.load(file)
for mr in merge_requests:
    # Get the author email from the merge request
    githubuser = username_mapping.get(mr.author['username'])
    print(githubuser)
    # Create the pull request on GitHub

   
    # Check if the pull request already exists
    existing_prs = github_repo.get_pulls(state='open', head=f"{user.login}:{mr.source_branch}", base=mr.target_branch)
    if existing_prs.totalCount > 0:
        github_pr = existing_prs[0]
        github_pr.edit(title=mr.title, body=mr.description)
        print(f"Updated existing pull request: {github_pr.html_url}")
    else:
        # Create the pull request on GitHub
        github_pr = github_repo.create_pull(
            title=mr.title,
            body=mr.description,
            head=mr.source_branch,
            base=mr.target_branch,
            maintainer_can_modify=True
        )
        print(f"Created new pull request: {github_pr.html_url}")
    # Migrate comments  
    discussions = mr.discussions.list()  
    for discussion in discussions:  
        first_comment = True 
        first_comment_id = None 
        f_comment_obj = None
        for note in discussion.attributes['notes']: 
            print(note) 
            author_name = note["author"]['name']  
            author_username = note["author"]['username']  
            comment_body = note['body']  
            if first_comment:  
                # Create the initial comment  
                comment = github_pr.create_issue_comment(f"{author_name} ({author_username}): {comment_body}")  
                first_comment = False
                first_comment_id = comment.id  
                f_comment_obj = comment_body

            else:  
                # Reply to the previous comment  
                print(first_comment_id)
                reply_body = f"> {f_comment_obj} \n\n {comment_body}"  
                comment = github_pr.create_issue_comment(body=reply_body)
  
print("Migration completed successfully.") 

ivbeljan_microsoft
Updated existing pull request: https://github.com/ivbeljan_microsoft/mynetcoreproject/pull/1
{'id': 2135348606, 'type': None, 'body': 'requested review from @beljan.ivan', 'attachment': None, 'author': {'id': 2862849, 'username': 'beljan.ivan', 'name': 'Ivan Beljan', 'state': 'active', 'locked': False, 'avatar_url': 'https://secure.gravatar.com/avatar/723bc1dfb00da3be0eb4bfe6366d599eb986e6b10812c95a757ffb62b9be8cb2?s=80&d=identicon', 'web_url': 'https://gitlab.com/beljan.ivan'}, 'created_at': '2024-09-30T09:22:35.283Z', 'updated_at': '2024-09-30T09:22:35.289Z', 'system': True, 'noteable_id': 330826202, 'noteable_type': 'MergeRequest', 'project_id': 62123776, 'resolvable': False, 'confidential': False, 'internal': False, 'imported': False, 'imported_from': 'none', 'noteable_iid': 1, 'commands_changes': {}}
{'id': 2135348608, 'type': None, 'body': 'assigned to @beljan.ivan', 'attachment': None, 'author': {'id': 2862849, 'username': 'beljan.ivan', 'name': 'Ivan Beljan',

In [104]:
print(first_comment_id)
#cooment = github_pr.get_comment(2420468416)
comment = github_pr.create_review_comment_reply(2420468416, "test")
print(comment)

2420468416


GithubException: 404 {"message": "Parent comment not found", "documentation_url": "https://docs.github.com/rest/pulls/comments#create-a-reply-for-a-review-comment", "status": "404"}